<a href="https://colab.research.google.com/github/thd2020/text-2-clip/blob/main/text_to_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py
# For security purposes, please check the contents of collect_env.py before running it.
!python collect_env.py

--2023-11-09 07:34:04--  https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21864 (21K) [text/plain]
Saving to: ‘collect_env.py’

collect_env.py      100%[===================>]  21.35K  --.-KB/s    in 0s      

2023-11-09 07:34:05 (126 MB/s) - ‘collect_env.py’ saved [21864/21864]

PyTorch version: 2.1.0+cu118
Is debug build: False
CUDA used to build PyTorch: 11.8
ROCM used to build PyTorch: N/A

OS: Ubuntu 22.04.2 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: 14.0.0-1ubuntu1.1
CMake version: version 3.27.7
Libc version: glibc-2.35

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] (64-bit runtime)
Python platform: Linux-5.15.120+

In [ ]:
!pip install pytorch-pretrained-biggan
!pip uninstall huggingface_hub
!pip install huggingface_hub
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.0
    Uninstalling huggingface-hub-0.19.0:
      Successfully uninstalled huggingface-hub-0.19.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.2 MB/s eta 0:00:00


In [ ]:
# 导入所需的库
import torch
import torch.hub
import torchvision
import torchaudio
import numpy as np
import cv2
import moviepy.editor as mpy
import os
import requests, json
import nltk.stem.wordnet
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal, convert_to_images)
from google.colab.patches import cv2_imshow

In [ ]:
!python3 -m nltk.downloader wordnet
!unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
Archive:  /root/nltk_data/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wo

In [ ]:
# 定义使用自然语言处理（NLP）模型提取文本中的关键词和类别的函数
def extract_keywords_and_category_with_nlp(text):
  # 加载预训练的BERT模型
  model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')
  # 加载预训练的BERT分词器
  tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
  # 设置模型为评估模式
  model.eval()
  # 将文本转换为张量
  tokens = tokenizer.encode(text, add_special_tokens=True)
  tokens = torch.LongTensor(tokens).unsqueeze(0)
  # 创建一个空的关键词列表
  keywords = []
  # 创建一个空的类别列表
  categories = []
  # 对每个单词进行处理
  for i in range(len(tokens[0])):
    # 获取单词的文本
    word = tokenizer.decode(tokens[0][i])
    print(word)
    # 如果单词是一个名词或形容词，且不是一个停用词，那么将其视为一个关键词
    if word in imagenet:
      # 将单词添加到关键词列表中
      keywords.append(word)
  return keywords

In [ ]:
# 定义使用BigGAN生成图像的函数
def generate_image_with_biggan(text):
  images=[]
  # 加载预训练的BigGAN模型
  model = BigGAN.from_pretrained('biggan-deep-512')
  # 设置模型为评估模式
  model.eval()
  # 使用自然语言处理（NLP）模型提取文本中的关键词
  keywords = extract_keywords_and_category_with_nlp(text)

  # Prepare a input
  truncation = 0.4
  print("keywords= ", keywords)
  class_vector = one_hot_from_names(keywords, batch_size=5)
  print("keywords = ", keywords, ", class_vertor = ", class_vector)
  noise_vector = truncated_noise_sample(truncation=truncation, batch_size=5)

  # All in tensors
  noise_vector = torch.from_numpy(noise_vector)
  class_vector = torch.from_numpy(class_vector)

  # If you have a GPU, put everything on cuda
  noise_vector = noise_vector.to('cuda')
  class_vector = class_vector.to('cuda')
  model.to('cuda')

  # Generate an image
  with torch.no_grad():
      output = model(noise_vector, class_vector, truncation)

  # If you have a GPU put back on CPU
  output = output.to('cpu')

  images = convert_to_images(output)

  # 返回生成的图像
  return images

In [ ]:
# 定义使用Tacotron 2生成音频的函数
def generate_audio_with_tacotron2(text):
  # 加载预训练的Tacotron 2模型
  model = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2', pretrained=True)
  # 设置模型为评估模式
  model.eval()
  # 加载预训练的WaveGlow模型
  waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow', pretrained=True)
  # 设置模型为评估模式
  waveglow.eval()
  # 将文本转换为张量
  text = torch.LongTensor(model.text_to_sequence(text, ['english_cleaners']))
  # 使用Tacotron 2模型生成音频的梅尔频谱
  with torch.no_grad():
    mel, _, _ = model.infer(text.unsqueeze(0))
  # 使用WaveGlow模型生成音频的波形
  with torch.no_grad():
    audio = waveglow.infer(mel)
  # 将音频转换为numpy数组
  audio = audio[0].detach().numpy()
  # 返回生成的音频
  return audio

In [ ]:
# 定义使用Video GPT生成视频的函数
def generate_video_with_videogpt(image, audio):
  # 加载预训练的Video GPT模型
  model = torch.hub.load('williamFalcon/pytorch-video-gan', 'videogan', pretrained=True)
  # 设置模型为评估模式
  model.eval()
  # 将图像和音频转换为张量
  images = torch.from_numpy(images).permute(0, 3, 1, 2)
  audio = torch.from_numpy(audio).unsqueeze(0)
  # 使用Video GPT模型生成视频
  with torch.no_grad():
    video = model(images, audio)
  # 将视频转换为numpy数组
  video = video[0].permute(1, 2, 3, 0).detach().numpy()
  # 返回生成的视频
  return video

In [ ]:
# 定义文本转视频的函数
def text_to_video(text):
  # 将文本分割成句子
  sentences = text.split(".")
  # 创建一个空的视频列表
  video_list = []
  # 对每个句子进行处理
  for sentence in sentences:
    # 使用BigGAN生成图像
    image = generate_image_with_biggan(sentence)
    # 使用Tacotron 2生成音频
    audio = generate_audio_with_tacotron2(sentence)
    # 使用Video GPT生成视频
    video = generate_video_with_videogpt(image, audio)
    # 将视频添加到视频列表中
    video_list.append(video)
  # 将视频列表拼接成一个完整的视频
  final_video = mpy.concatenate_videoclips(video_list)
  # 返回最终的视频
  return final_video

In [ ]:
# 定义要打开的文本文件的路径
file_path = "imagenet_classes.txt"

# 以读取模式打开文本文件
with open(file_path, "r") as file:
    # 读取文本文件的内容
    content = file.read()
    imagenet = content.replace("\n", ",").replace("\n","").replace(" ","").split(",")
    imagenet = [x.replace("\n","") for x in imagenet]

In [112]:
erer = "mink fly bee pug chow"


images = generate_image_with_biggan(erer)

for image in images:
  img = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
  cv2_imshow(img)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


[CLS]
min
##k
fly
bee
pu
##g
ch
##ow
[SEP]
keywords=  ['fly', 'bee']


AssertionError: ignored

In [ ]:
# 定义一个示例文本
text = "cat dog."

# 调用文本转视频的函数
video = text_to_video(text)

# 保存视频到本地
video.write_videofile("text_to_video.mp4")


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


[CLS]
cat
dog
[SEP]
keywords=  []


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


AssertionError: ignored